# Show "best fit" high-pressure limit kinetics 
Run this script once to identify a list of reactions including indices that produce the highest 
rates per reaction family for unimolecular and bimolecular reactions. Based on the result, 
improve the database and run the script again. Repeat this process until the "best fit" 
high-pressure limit kinetics show "reasonable" rates.

In [ ]:
import os
import time
import operator
import unittest 
import numpy as np
from copy import deepcopy

from rmgpy import settings
from rmgpy.data.kinetics.database import KineticsDatabase
from rmgpy.data.base import DatabaseError
from rmgpy.molecule.molecule import Molecule
from rmgpy.species import Species
from rmgpy.data.rmg import RMGDatabase
from rmgpy.data.kinetics.common import KineticsError, saveEntry
from rmgpy.kinetics.arrhenius import Arrhenius
from rmgpy.data.rmg import getDB
from rmgpy.thermo.thermoengine import submit
from rmgpy.reaction import Reaction
import matplotlib.pyplot as plt
%matplotlib inline

## Step 1: 
Select the family to work on.

In [ ]:
family = 'Intra_R_Add_Exocyclic'

In [ ]:
#path = os.path.join(settings['database.directory'], 'kinetics','families')
database = RMGDatabase()
database.load(
    settings['database.directory'], 
    thermoLibraries = ['primaryThermoLibrary','Klippenstein_Glarborg2016','BurkeH2O2','thermo_DFT_CCSDTF12_BAC','CBS_QB3_1dHR', 'DFT_QCI_thermo'],
    transportLibraries = [],
    reactionLibraries = [],
    seedMechanisms = [],
    kineticsFamilies = [family],
    kineticsDepositories = ['training'],
    depository = False,  
)

In [ ]:
#all_families = getDB('kinetics').families.keys()
#all_families = ['H_Abstraction']

# Discrete temperature range
Ts = np.linspace(300,2000,30)

In [ ]:
def analyze_reactions(fam_name, molecularity=1, forward=True):
    print fam_name
    
    fam = database.kinetics.families[fam_name]
    dep = fam.getTrainingDepository()
    rxns = []
    list_indices = []

    # Extract all training reactions for selected family
    for entry in dep.entries.values():
        r = entry.item
        r.kinetics = entry.data
        r.index = entry.index
        for spc in r.reactants+r.products:
            if spc.thermo is None:
                submit(spc)
        rxns.append(r)

    # Only proceed if at least one training reaction is available
    if rxns:         
        # Get kinetic rates for unimolecular reactions
        k_list = []
        index_list = []
        for rxn in rxns:
            if forward:
                if len(rxn.reactants) == molecularity:
                    k_list.append(rxn.kinetics)
                    index_list.append(rxn.index)
            else:
                if len(rxn.products) == molecularity:
                    k_list.append(rxn.generateReverseRateCoefficient())
                    index_list.append(rxn.index)

        # Get max. kinetic rates at each discrete temperature
        if k_list:
            k_max_list = []
            rxn_indices = set()
            for T in Ts:
                mydict = {}
                kvals = [k.getRateCoefficient(T) for k in k_list]
                mydict = dict(zip(index_list, kvals))

                # Find key and value of max rate coefficient
                key_max_rate = max(mydict.iteritems(), key=operator.itemgetter(1))[0]
                rxn_indices.add(key_max_rate)
                kval = mydict[key_max_rate]
                k_max_list.append(kval)
                print """For {0} 1/K training reaction {1} with index {2} has the highest rate 
                of {3}.""".format(1000.0/T, dep.entries.get(key_max_rate), key_max_rate, kval)
                
                if molecularity == 2:
                    max_rxn = dep.entries.get(key_max_rate).item
                    display(max_rxn)
                    print "collision limit:  {}".format(max_rxn.calculate_coll_limit(T))
                    print "collision limit agrees?"
                    print [] == max_rxn.check_collision_limit_violation( 300.0, 2000.0, 0.1, 100.0)


            units = 's^-1' if molecularity == 1 else 'm^3/(mol*s)'
                
            arr = Arrhenius().fitToData(Ts,np.array(k_max_list), units)

            fig = plt.figure()
            fig_name = fam_name
            fig_name += ' Unimolecular' if molecularity == 1 else ' Bimolecular'
            #save_path = 'ArrheniusFig/'
            plt.semilogy(1000.0/Ts, k_max_list, label=fig_name)
            plt.xlabel("1000/T (1/K)")
            plt.ylabel("k ({0})".format(units))
            plt.legend(loc='upper left')
            #fig.savefig((save_path + fig_name + '_Unimolecular' + '.png'), bbox_inches='tight')
            #plt.close("all")
            
            if not forward:
                spc_set = set()
                for rxn_ind in rxn_indices:
                    spc_set.update(rxns[rxn_ind].reactants)
                    spc_set.update(rxns[rxn_ind].products)
                for spcs in spc_set:
                    display(spcs)
                    print '{0}: {1} (kJ/mol): {2}'.format(spcs, spcs.thermo.getEnthalpy(298)/1000., spcs.thermo.comment)

## Step 2a: unimolecular reactions - forward
Generate a list of reactions including indices that produce the highest rates per reaction family for unimolecular reactions.

In [ ]:
analyze_reactions(family, molecularity=1, forward=True)

## Step 2a: unimolecular reactions - backward
Generate a list of reactions including indices that produce the highest rates per reaction family for unimolecular reactions.

In [ ]:
analyze_reactions(family, molecularity=1, forward=False)

## Step 2b: bimolecular reactions - forward 
Generate a list of reactions including indices that produce the highest rates per reaction family for bimolecular reactions.

In [ ]:
analyze_reactions(family, molecularity=2, forward=True)

## Step 2b: bimolecular reactions - backward
Generate a list of reactions including indices that produce the highest rates per reaction family for bimolecular reactions.

In [ ]:
analyze_reactions(family, molecularity=2, forward=False)